<a href="https://colab.research.google.com/github/Kishoreraj2024/POAI-EXP-6/blob/main/POAI_EXP_6ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from typing import Union, List, Optional, Dict

def unify(x: Union[str, List],
          y: Union[str, List],
          theta: Optional[Dict] = None) -> Optional[Dict]:
    """Unify two expressions with substitution theta.
    Returns substitution dict if successful, None otherwise."""
    if theta is None:
        theta = {}
    if x == y:
        return theta
    elif isinstance(x, str) and x.islower():  # x is variable
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():  # y is variable
        return unify_var(y, x, theta)
    elif isinstance(x, list) and isinstance(y, list) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], theta))
    else:
        return None

def unify_var(var: str,
              x: Union[str, List],
              theta: Dict) -> Optional[Dict]:
    """Unify a variable with a term."""
    if var in theta:
        return unify(theta[var], x, theta)
    elif isinstance(x, str) and x in theta:
        return unify(var, theta[x], theta)
    elif occurs_check(var, x, theta):
        return None
    else:
        # Copy theta to avoid mutation issues
        theta_ = theta.copy()
        theta_[var] = x
        return theta_

def occurs_check(var: str,
                 term: Union[str, List],
                 theta: Dict) -> bool:
    """Check if variable occurs in term to prevent infinite loops."""
    if var == term:
        return True
    elif isinstance(term, str) and term in theta:
        return occurs_check(var, theta[term], theta)
    elif isinstance(term, list):
        return any(occurs_check(var, subterm, theta) for subterm in term)
    else:
        return False

def substitute(expr: Union[str, List],
               theta: Dict) -> Union[str, List]:
    """Apply substitution to an expression."""
    if isinstance(expr, str):
        return theta.get(expr, expr)
    elif isinstance(expr, list):
        return [substitute(term, theta) for term in expr]
    else:
        return expr

def resolution(kb: Dict,
               query: Union[str, List]) -> bool:
    """
    Resolution algorithm for logical inference.
    Returns True if query can be proved, False otherwise.
    """
    # Check facts first
    for fact in kb['facts']:
        if unify(fact, query, {}):
            return True

    # Check rules
    for rule in kb['rules']:
        head, body = rule
        theta = unify(head, query, {})
        if theta is not None:
            # Prove each subgoal with substitution
            if all(resolution(kb, substitute(subgoal, theta)) for subgoal in body):
                return True
    return False

def main():
    """Main execution"""
    # Knowledge Base
    knowledge_base = {
        'facts': [
            ['human', 'john']  # human(john)
        ],
        'rules': [
            # mortal(X) :- human(X)
            (['mortal', 'X'], [['human', 'X']])
        ]
    }

    # Query: Is John mortal?
    query = ['mortal', 'john']

    # Run resolution
    print("Knowledge Base:")
    print("- Facts:", knowledge_base['facts'])
    print("- Rules:", knowledge_base['rules'])
    print("\nQuery:", query)

    if resolution(knowledge_base, query):
        print("\nResult: SUCCESS - John is mortal!")
    else:
        print("\nResult: FAILED - Cannot prove John is mortal")

if __name__ == "__main__":
    main()


Knowledge Base:
- Facts: [['human', 'john']]
- Rules: [(['mortal', 'X'], [['human', 'X']])]

Query: ['mortal', 'john']

Result: SUCCESS - John is mortal!
